In [1]:
from huggingface_hub import login, HfFolder
login(token='', add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (manager,store).
Your token has been saved to C:\Users\Zhenya\.cache\huggingface\token
Login successful


In [3]:
import logging
import os
import re
import shutil
import sys
import time
from dataclasses import dataclass, field
from functools import partial
from pathlib import Path
from typing import Any, Dict, List, Optional, Union

import datasets
import evaluate
import numpy as np
import torch
import torch.nn as nn
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from datasets import (
    DatasetDict,
    IterableDataset,
    IterableDatasetDict,
    concatenate_datasets,
    interleave_datasets,
    load_dataset,
)
from huggingface_hub import Repository, create_repo
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AddedToken,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    WhisperConfig,
    WhisperFeatureExtractor,
    WhisperForConditionalGeneration,
    WhisperProcessor,
    WhisperTokenizerFast,
    get_scheduler,
    set_seed,
)
from transformers.modeling_outputs import BaseModelOutput
from transformers.models.whisper.english_normalizer import BasicTextNormalizer, EnglishTextNormalizer
from transformers.utils import check_min_version
from transformers.utils.versions import require_version


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.34.0.dev0")

require_version("datasets>=2.14.6", "To fix: `pip install --upgrade datasets`")

logger = get_logger(__name__)

In [4]:
from run_distillation import (ModelArguments, DataTrainingArguments, DistillationTrainingArguments,
    DataCollatorSpeechSeq2SeqWithPadding, log_metric, log_pred, convert_dataset_str_to_list,
    load_multiple_datasets, get_layers_to_supervise, sorted_checkpoints, rotate_checkpoints,
    get_last_checkpoint, get_parameter_names)

In [5]:
raw_datasets = DatasetDict()
# sampling_rate = 16_000
# 3. Load dataset
all_train_datasets_list = []
all_eval_datasets_list = []

# Mozila UK

def prepare_mozilla_uk_dataset(folder_path, split):
    split_dataset = datasets.load_from_disk(f'{folder_path}/{split}',) 
    split_dataset = split_dataset.rename_column("sentence", "text")
    split_dataset = split_dataset.remove_columns(
        set(split_dataset.features.keys()) - {"audio", "text", "whisper_transcript"}
        )
    return split_dataset

for split in ['train', 'validation']:
    print(split)
    all_train_datasets_list.append(
        prepare_mozilla_uk_dataset('dataset_saved/labeled', split)
        )
    
for split in ['test']:
    print(split)
    all_eval_datasets_list.append(
        prepare_mozilla_uk_dataset('dataset_saved/labeled', split)
        )
    
# Place for next dataset    

raw_datasets['train'] = concatenate_datasets(all_train_datasets_list)
raw_datasets['eval'] = concatenate_datasets(all_eval_datasets_list)

train
validation
test


In [7]:
# 1. Parse input arguments
# We keep distinct sets of args, for cleaner separation of model/data/training related args
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, DistillationTrainingArguments))

list_args = [
    '--model_name_or_path=./models_dir/student_moz_uk',
    # '--teacher_model_name_or_path=openai/whisper-medium',
    '--teacher_model_name_or_path=./local_whisper_medium',
    '--eval_steps=500',
    '--save_steps=500',
    '--warmup_steps=250',
    '--learning_rate=0.00001',
    '--lr_scheduler_type=constant_with_warmup',
    '--logging_steps=25',
    '--save_total_limit=3',
    '--max_steps=10000',
    # '--wer_threshold=None',
    '--per_device_train_batch_size=24',
    '--per_device_eval_batch_size=24',
    '--dataloader_num_workers=4',
    '--preprocessing_num_workers=4',
    '--ddp_timeout=7200',
    '--dtype=float16',
    '--do_train=True',
    '--do_eval=True',
    '--gradient_checkpointing=True',
    '--streaming=False',
    '--cache_dir=./model_cache/',

    '--overwrite_output_dir=True',
    '--output_dir=./result_distiling_3',
    '--freeze_encoder=True',
    '--language=uk',
    # '--=',
]

model_args, data_args, training_args = parser.parse_args_into_dataclasses(list_args)

In [8]:
# 2. Initialize the accelerator
# We will let the accelerator handle device placement for us in this example
# We simply have to specify the training precision and any trackers being used
# We'll use the same dtype arguments as our JAX/Flax training script and convert
# it to accelerate format
# The teacher model can safely be cast to the dtype of training since we don't
# update the params
if training_args.dtype == "float16":
    mixed_precision = "fp16"
    teacher_dtype = torch.float16
elif training_args.dtype == "bfloat16":
    mixed_precision = "bf16"
    teacher_dtype = torch.bfloat16
else:
    mixed_precision = "no"
    teacher_dtype = torch.float32

accelerator = Accelerator(
    gradient_accumulation_steps=training_args.gradient_accumulation_steps,
    mixed_precision=mixed_precision,
    log_with=training_args.report_to,
    project_dir=training_args.output_dir,
)

accelerator.init_trackers(project_name=data_args.wandb_project)

# 3. Set-up basic logging
# Create one log on every process with the configuration for debugging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
# Log a small summary on each proces
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}, "
    f"distributed training: {training_args.parallel_mode.value == 'distributed'}, 16-bits training: {training_args.fp16}"
)

# Set the verbosity to info of the Transformers logger (on main process only)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()
logger.info("Training/evaluation parameters %s", training_args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zekamrozek. Use `wandb login --relogin` to force relogin


04/16/2024 18:11:57 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/16/2024 18:11:57 - INFO - __main__ - Training/evaluation parameters DistillationTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=4,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=7200,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=True,
dtype=float16,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500.0,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
freeze_encoder=True,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},


In [9]:
last_checkpoint = None

In [10]:
# 5. Handle the repository creation
if accelerator.is_main_process:
    if training_args.push_to_hub:
        # Retrieve of infer repo_name
        repo_name = training_args.hub_model_id
        if repo_name is None:
            repo_name = Path(training_args.output_dir).absolute().name
        # Create repo and retrieve repo_id
        repo_id = create_repo(repo_name, exist_ok=True, token=training_args.hub_token).repo_id
        # Clone repo locally
        repo = Repository(training_args.output_dir, clone_from=repo_id, token=training_args.hub_token)

        with open(os.path.join(training_args.output_dir, ".gitignore"), "w+") as gitignore:
            if "wandb" not in gitignore:
                gitignore.write("wandb\n")
    elif training_args.output_dir is not None:
        os.makedirs(training_args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [11]:
# 7. Load pretrained model, tokenizer, and feature extractor
config = WhisperConfig.from_pretrained(
    (model_args.config_name if model_args.config_name else model_args.model_name_or_path),
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    token=model_args.token,
)
feature_extractor = WhisperFeatureExtractor.from_pretrained(
    (model_args.feature_extractor_name if model_args.feature_extractor_name else model_args.model_name_or_path),
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    token=model_args.token,
)
tokenizer = WhisperTokenizerFast.from_pretrained(
    (model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path),
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    token=model_args.token,
)

# override timestamp tokens until tokenizer issues are fixed in transformers
timestamps = [AddedToken("<|%.2f|>" % (i * 0.02), lstrip=False, rstrip=False) for i in range(1500 + 1)]
tokenizer.add_tokens(timestamps)


loading configuration file ./models_dir/student_moz_uk\config.json
Model config WhisperConfig {
  "_name_or_path": "./local_whisper_medium",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 2,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 24,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "

0

In [12]:
teacher_model = WhisperForConditionalGeneration.from_pretrained(
    model_args.teacher_model_name_or_path,
    cache_dir=model_args.cache_dir,
    token=model_args.token,
    low_cpu_mem_usage=True,
    torch_dtype=teacher_dtype,
)

# teacher_model.save_pretrained('./local_whisper_medium')

loading configuration file ./local_whisper_medium\config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-medium",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "apply_spec_augment": false,
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "classifier_proj_size": 256,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 24,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 24,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_

In [13]:
student_model = WhisperForConditionalGeneration.from_pretrained(
    model_args.model_name_or_path,
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    subfolder=model_args.subfolder,
    token=model_args.token,
    low_cpu_mem_usage=True,
)

loading weights file ./models_dir/student_moz_uk\model.safetensors
Generate config GenerationConfig {
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "decoder_start_token_id": 50258,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "max_length": 448,
  "pad_token_id": 50257,
  "suppress_tokens": [
    1,
    2,
    7,
    8,
    9,
    10,
    14,
    25,
    26,
    27,
    28,
    29,
    31,
    58,
    59,
    60,
    61,
    62,
    63,
    90,
    91,
    92,
    93,
    359,
    503,
    522,
    542,
    873,
    893,
    902,
    918,
    922,
    931,
    1350,
    1853,
    1982,
    2460,
    2627,
    3246,
    3253,
    3268,
    3536,
    3846,
    3961,
    4183,
    4667,
    6585,
    6647,
    7273,
    9061,
    9383,
    10428,
    10929,
    11938,
    12033,
    12331,
    12562,
    13793,
    14157,
    14635,
    15265,


In [14]:
if student_model.config.decoder_start_token_id is None or teacher_model.config.decoder_start_token_id is None:
    raise ValueError(
        f"Make sure that `config.decoder_start_token_id` is correctly defined for both the "
        f"student and teacher model. Got {student_model.config.decoder_start_token_id} for the "
        f"student and {teacher_model.config.decoder_start_token_id} for the teacher."
    )

share_hidden_states = training_args.freeze_encoder and student_model.config.d_model == teacher_model.config.d_model

# enable gradient checkpointing if necessary
if training_args.gradient_checkpointing:
    student_model.gradient_checkpointing_enable()

# freeze student encoder if necessary
if training_args.freeze_encoder:
    student_model.freeze_encoder()
    student_model.model.encoder.gradient_checkpointing = False

# if share_hidden_states:
#     # tie the weights for the teacher encoder if we're freezing the student and it's the same as the teacher
#     teacher_model.model.encoder = student_model.model.encoder

is_multilingual = False

In [15]:
# 8. Create a single speech processor - make sure all processes wait until data is saved
if accelerator.is_main_process:
    feature_extractor.save_pretrained(training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    # save the config and generation config as well
    config.save_pretrained(training_args.output_dir)
    student_model.generation_config.save_pretrained(training_args.output_dir)

accelerator.wait_for_everyone()

Feature extractor saved in ./result_distiling_3\preprocessor_config.json
tokenizer config file saved in ./result_distiling_3\tokenizer_config.json
Special tokens file saved in ./result_distiling_3\special_tokens_map.json
Configuration saved in ./result_distiling_3\config.json
Configuration saved in ./result_distiling_3\generation_config.json


In [16]:
training_args.output_dir

'./result_distiling_3'

In [17]:
processor = WhisperProcessor.from_pretrained(training_args.output_dir)

loading configuration file ./result_distiling_3\preprocessor_config.json
Feature extractor WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

loading file vocab.json
loading file tokenizer.json
loading file merges.txt
loading file normalizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# 9. Resample speech dataset: `datasets` takes care of automatically loading and resampling the audio,
# so we just need to set the correct target sampling rate.
sampling_rate = feature_extractor.sampling_rate
raw_datasets = raw_datasets.cast_column(
    data_args.audio_column_name,
    datasets.features.Audio(sampling_rate=sampling_rate),
)

In [19]:
# 10. Preprocessing the datasets: we need to read the audio files as arrays and tokenize the targets.
# 10.1: Define the pre-processing constants
max_input_length = int(data_args.max_duration_in_seconds * sampling_rate)
min_input_length = int(data_args.min_duration_in_seconds * sampling_rate)
max_label_length = (
    data_args.max_label_length if data_args.max_label_length is not None else student_model.config.max_length
)

timestamp_probability = data_args.timestamp_probability
condition_on_prev_probability = data_args.condition_on_prev_probability
return_timestamps = data_args.return_timestamps if timestamp_probability > 0 else False

timestamp_ids = tokenizer.timestamp_ids()
timestamp_begin = tokenizer.all_special_ids[-1]
timestamp_position = 3 if is_multilingual else 1

decoder_start_token_id = student_model.config.decoder_start_token_id  # <|startoftranscript|>
decoder_prev_token_id = tokenizer.all_special_ids[-3]  # <|startofprev|>
decoder_eot_token_id = tokenizer.eos_token_id

language = data_args.language
task = data_args.task

num_workers = data_args.preprocessing_num_workers
dataloader_num_workers = training_args.dataloader_num_workers

metric = evaluate.load("wer")
normalizer = (
    BasicTextNormalizer() if language is not None else EnglishTextNormalizer(tokenizer.english_spelling_normalizer)
)

In [20]:
wer_threshold = 10#data_args.wer_threshold


# 10.3: filter training data based on WER threshold -> this is KEY to good distillation performance
def is_wer_in_range(ground_truth, whisper_transcript):
    norm_ground_truth = normalizer(ground_truth)
    if (
        isinstance(whisper_transcript, str)
        and whisper_transcript.startswith("[")
        and whisper_transcript.endswith("]")
    ):
        whisper_transcript = re.findall(r"\d+", whisper_transcript)
        whisper_transcript = [int(token) for token in whisper_transcript]
    if isinstance(whisper_transcript, list):
        whisper_transcript = tokenizer.decode(whisper_transcript, skip_special_tokens=True)
    if len(norm_ground_truth) > 0 and whisper_transcript is not None:
        norm_whisper_transcript = normalizer(whisper_transcript)
        wer = 100 * metric.compute(predictions=[norm_whisper_transcript], references=[norm_ground_truth])
        return wer < wer_threshold
    else:
        # filter automatically since we can't know the WER
        return False

In [21]:
print(raw_datasets['train'].num_rows, raw_datasets['eval'].num_rows)

for split_for_wer_filter in [
    'train', 'eval'
]:
    print(split_for_wer_filter
    )
    filter_by_wer_threshold = partial(
        raw_datasets[split_for_wer_filter].filter,
        function=is_wer_in_range,
        input_columns=["text", "whisper_transcript"],
    )

    if wer_threshold is not None:
        raw_datasets[split_for_wer_filter] = filter_by_wer_threshold(num_proc=1, desc="filtering train dataset by wer")
    
print(raw_datasets['train'].num_rows, raw_datasets['eval'].num_rows)

33585 9791
train


filtering train dataset by wer:   0%|          | 0/33585 [00:00<?, ? examples/s]

eval


filtering train dataset by wer:   0%|          | 0/9791 [00:00<?, ? examples/s]

11439 3196


In [22]:
# 10.4: pre-process training/evaluation datasets
def has_timestamp_tokens(input_str):
    """
    Identify whether the input string contains timestamp tokens, of the form <|0.00|>, by searching for
    pairs of left and right-angle brackets.
    """
    return bool(re.search("\<[^\>]*\>", input_str))

def prepare_train_dataset(batch):
    """
    Pre-process the raw dataset in a three stage process:
        1. Convert the audio arrays to log-mel spectrogram inputs
        2. Possibly filter the timestamp tokens from the token ids (depending on the timestamp probability)
        3. Possibly add prompt tokens if conditioning on previous text (depending on the conditioning probability)
    TODO(SG): see whether we can 'pack' the audio inputs closer to 30 second chunks
    """
    # process audio input
    audio = [sample["array"] for sample in batch["audio"]]
    inputs = feature_extractor(audio, sampling_rate=sampling_rate)
    batch["input_features"] = inputs.input_features
    batch["input_length"] = [len(sample) for sample in audio]

    # process text targets - for training these are the Whisper-generated pseudo-labels
    input_str_batched = batch["whisper_transcript"]

    all_token_ids = []
    all_token_ids_unprompted = []
    for input_str in input_str_batched:
        if isinstance(input_str, list):
            # pseudo-labelled transcriptions have been retained as token ids (`decode_token_ids=False`)
            token_ids = input_str
        elif input_str[0].startswith("[") and input_str[0].endswith("]"):
            token_ids = re.findall(r"\d+", input_str)
            token_ids = [int(token) for token in token_ids]
        else:
            token_ids = None

        if token_ids is not None:
            # remove the EOT tokens to get the 'true' token length
            token_ids = [token for token in token_ids if token != decoder_eot_token_id]
            token_ids = token_ids + [decoder_eot_token_id]
            # check whether we have timestamps in the PLs and filter if required
            has_timestamps = len(set(token_ids) & set(timestamp_ids)) > 0
            if has_timestamps:
                # sample from binomial distribution to get probability of training on timestamps
                predict_timestamps = bool(np.random.binomial(1, timestamp_probability))
                if not predict_timestamps:
                    # filter timestamps and insert the <|notimestamps|> task token
                    token_ids = [token for token in token_ids if token < timestamp_begin]
                    token_ids.insert(timestamp_position, timestamp_begin)
        else:
            # pseudo-labelled transcriptions have been decoded to text (`decode_token_ids=True`)
            has_timestamps = has_timestamp_tokens(input_str)

            if has_timestamps:
                predict_timestamps = bool(np.random.binomial(1, timestamp_probability))
                if not predict_timestamps:
                    # filter timestamp token ids if not part of the prediction task
                    input_str = tokenizer._filter_timestamp_ids(input_str)
            else:
                predict_timestamps = False

            tokenizer.set_prefix_tokens(language=language, task=task, predict_timestamps=predict_timestamps)
            token_ids = tokenizer(input_str).input_ids

        all_token_ids_unprompted.append(token_ids)
        # check whether to condition on previous text - we do this with probability condition_on_prev_probability
        condition_on_prev = bool(np.random.binomial(1, condition_on_prev_probability))
        if condition_on_prev and len(all_token_ids_unprompted) > 1:
            # prompt ids are the penultimate token ids in the batch
            prompt_ids = all_token_ids_unprompted[-2]
            # strip timestamp tokens from prompt
            prompt_ids = [token for token in prompt_ids if token < timestamp_begin]
            if len(prompt_ids) > 0:
                # remove the standard task tokens and add the special <|startofprev|> token
                prompt_ids = [decoder_prev_token_id] + prompt_ids[timestamp_position:-1]
            if len(prompt_ids + token_ids) < max_label_length:
                token_ids = prompt_ids + token_ids
        all_token_ids.append(token_ids)

    batch["labels"] = all_token_ids
    return batch

def prepare_eval_dataset(batch):
    # process audio input
    sample = batch["audio"]
    inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])
    batch["input_features"] = inputs.input_features[0]
    batch["input_length"] = len(sample["array"])

    # process targets - for evaluation these are the ground-truth transcriptions
    input_str = batch["text"]
    batch["labels"] = tokenizer(input_str).input_ids
    return batch

In [23]:
vectorized_datasets = DatasetDict()
raw_datasets_train_features = list(raw_datasets["train"].features.keys())

map_fn_train = partial(
    raw_datasets['train'].map,
    function=prepare_train_dataset,
    remove_columns=raw_datasets_train_features,
    batched=True,
    batch_size=max(training_args.per_device_train_batch_size // 4, 4),  # TODO(SG) make data prep bs configurable
)
vectorized_datasets["train"] = map_fn_train(num_proc=1, desc="preprocess train dataset")


raw_datasets_eval_features = list(raw_datasets['eval'].features.keys())
map_fn_eval = partial(
    raw_datasets['eval'].map, function=prepare_eval_dataset, remove_columns=raw_datasets_eval_features
)

vectorized_datasets['eval'] = map_fn_eval(num_proc=1, desc="preprocess eval dataset")

preprocess train dataset:   0%|          | 0/11439 [00:00<?, ? examples/s]

preprocess eval dataset:   0%|          | 0/3196 [00:00<?, ? examples/s]

In [24]:
# 10.5: Filter training data with inputs longer than `max_input_length`
def is_audio_in_length_range(length):
    return min_input_length < length < max_input_length

filter_by_audio_fn = partial(
    vectorized_datasets.filter, function=is_audio_in_length_range, input_columns=["input_length"]
)
vectorized_datasets = filter_by_audio_fn(num_proc=1, desc="filtering train dataset by audio length")

# 10.6: Filter training data with labels longer than `max_label_length`
def is_labels_in_length_range(labels):
    return 0 < len(labels) <= max_label_length

filter_by_labels_fn = partial(
    vectorized_datasets.filter, function=is_labels_in_length_range, input_columns=["labels"]
)
vectorized_datasets = filter_by_labels_fn(num_proc=1, desc="filtering train dataset")

# Pre-processing complete!
# For large datasets it is advised to run the preprocessing on a
# single machine first with `--preprocessing_only` since there will mostly likely
# be a timeout when running the script in distributed mode.
# In a second step, `--preprocessing_only` can then be set to `False` to load the
# cached dataset

filtering train dataset by audio length:   0%|          | 0/11439 [00:00<?, ? examples/s]

filtering train dataset by audio length:   0%|          | 0/3196 [00:00<?, ? examples/s]

filtering train dataset:   0%|          | 0/11439 [00:00<?, ? examples/s]

filtering train dataset:   0%|          | 0/3196 [00:00<?, ? examples/s]

In [25]:
cache = {k: v.cache_files for k, v in vectorized_datasets.items()}
logger.info(f"Data preprocessing finished. Files cached at {cache}.")


04/16/2024 18:26:39 - INFO - __main__ - Data preprocessing finished. Files cached at {'train': [{'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\train\\cache-986c887d0bd1f317.arrow'}, {'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\train\\cache-cef3de1d3b821f5b.arrow'}], 'eval': [{'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\test\\cache-905c92fa89753543.arrow'}, {'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\test\\cache-d290041596bd97bc.arrow'}]}.


In [26]:
# cache
{'train': [{'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\train\\cache-5098ea3880ff3c68.arrow'},
  {'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\train\\cache-72585aa289e41103.arrow'}],
 'eval': [{'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\test\\cache-e7d01e48949f8d93.arrow'},
  {'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\test\\cache-2e2173af0f196642.arrow'}]}

{'train': [{'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\train\\cache-5098ea3880ff3c68.arrow'},
  {'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\train\\cache-72585aa289e41103.arrow'}],
 'eval': [{'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\test\\cache-e7d01e48949f8d93.arrow'},
  {'filename': 'f:\\distiling_whisper_local\\dataset_saved\\labeled\\test\\cache-2e2173af0f196642.arrow'}]}

In [27]:
# 11. Define Evaluation Metrics
def compute_metrics(preds, labels):
    # replace padded labels by the padding token
    for idx in range(len(labels)):
        labels[idx][labels[idx] == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True, decode_with_timestamps=return_timestamps)
    # we do not want to group tokens when computing the metrics
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # normalize everything and re-compute the WER
    norm_pred_str = [normalizer(pred) for pred in pred_str]
    norm_label_str = [normalizer(label) for label in label_str]
    # for logging, we need the pred/labels to match the norm_pred/norm_labels, so discard any filtered samples here
    pred_str = [pred_str[i] for i in range(len(norm_pred_str)) if len(norm_label_str[i]) > 0]
    label_str = [label_str[i] for i in range(len(norm_label_str)) if len(norm_label_str[i]) > 0]
    # filtering step to only evaluate the samples that correspond to non-zero normalized references:
    norm_pred_str = [norm_pred_str[i] for i in range(len(norm_pred_str)) if len(norm_label_str[i]) > 0]
    norm_label_str = [norm_label_str[i] for i in range(len(norm_label_str)) if len(norm_label_str[i]) > 0]

    wer = 100 * metric.compute(predictions=norm_pred_str, references=norm_label_str)
    return {"wer": wer, "wer_ortho": wer_ortho}, pred_str, label_str, norm_pred_str, norm_label_str

In [28]:
# 12. Define Training Schedule
# Store some constants
per_device_train_batch_size = int(training_args.per_device_train_batch_size)
train_batch_size = per_device_train_batch_size * accelerator.num_processes
gradient_accumulation_steps = int(training_args.gradient_accumulation_steps)
per_device_eval_batch_size = int(training_args.per_device_eval_batch_size)

if not data_args.streaming and training_args.max_steps < 0:
    num_epochs = int(training_args.num_train_epochs)
    steps_per_epoch = len(vectorized_datasets["train"]) // (train_batch_size * gradient_accumulation_steps)
    total_train_steps = steps_per_epoch * num_epochs
elif training_args.max_steps > 0:
    logger.info("max_steps is given, it will override any value given in num_train_epochs")
    total_train_steps = int(training_args.max_steps)
    # Setting a very large number of epochs so we go as many times as necessary over the iterator.
    num_epochs = sys.maxsize
    steps_per_epoch = total_train_steps
else:
    raise ValueError("max_steps must be specified when training with a streaming (iterable) dataset")

if training_args.eval_steps is None:
    logger.info(
        f"eval_steps is not set, evaluating at the end of {'each epoch' if not data_args.streaming else 'training'}"
    )
    eval_steps = steps_per_epoch
else:
    eval_steps = training_args.eval_steps

04/16/2024 18:26:39 - INFO - __main__ - max_steps is given, it will override any value given in num_train_epochs


In [29]:
# 13. Define optimizer, LR scheduler, collator
decay_parameters = get_parameter_names(
    student_model,
    [nn.LayerNorm],
    forbidden_module=[student_model.model.encoder] if training_args.freeze_encoder else None,
)
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [param for name, param in student_model.named_parameters() if name in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [param for name, param in student_model.named_parameters() if name not in decay_parameters],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(
    params=optimizer_grouped_parameters,
    lr=training_args.learning_rate,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
)

# LR scheduler gets stepped by `num_processes` each time -> account for this in warmup / total steps
lr_scheduler = get_scheduler(
    name=training_args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=training_args.warmup_steps * accelerator.num_processes,
    num_training_steps=total_train_steps * accelerator.num_processes,
)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=decoder_start_token_id,
    decoder_prev_token_id=decoder_prev_token_id,
    input_padding="longest",
    target_padding="max_length",
    max_target_length=max_label_length,
)

In [30]:
# 14. Define generation arguments - we need to do this before we wrap the models in DDP
# so that we can still access the configs
num_beams = (
    training_args.generation_num_beams
    if training_args.generation_num_beams is not None
    else getattr(student_model.generation_config, "num_beams", 1)
)

gen_kwargs = {
    "max_length": max_label_length,
    "num_beams": num_beams,
    "return_timestamps": return_timestamps,
}
if hasattr(teacher_model.generation_config, "is_multilingual") and teacher_model.generation_config.is_multilingual:
    # forcing the language and task tokens helps multilingual models in their generations
    gen_kwargs.update(
        {
            "language": data_args.language,
            "task": data_args.task,
        }
    )

In [31]:
# 15. Prepare everything with accelerate
student_model, teacher_model, optimizer, lr_scheduler = accelerator.prepare(
    student_model, teacher_model, optimizer, lr_scheduler
)


In [32]:

def kl_divergence(target_distribution, log_predicted_distribution, labels):
    kl_loss = nn.KLDivLoss(reduction="none")
    divergence = kl_loss(log_predicted_distribution, target_distribution)
    # ignore padded tokens from divergence, i.e. where labels are not set to -100
    padding_mask = labels >= 0
    padding_mask = padding_mask.unsqueeze(-1)
    divergence = divergence * padding_mask
    # take the average over the mini-batch
    divergence = divergence.sum() / padding_mask.sum()
    return divergence

# Define gradient update step fn
def train_step(
    batch,
    temperature=2.0,
):
    student_model.train()
    teacher_model.eval()

    student_outputs = student_model(**batch)
    with torch.no_grad():
        if share_hidden_states:
            # if the student and teacher share the same frozen encoder then we don't have to recompute the
            # encoder hidden-states for the teacher model, we can just re-use from the student
            encoder_outputs = BaseModelOutput(student_outputs.encoder_last_hidden_state)
            teacher_outputs = teacher_model(encoder_outputs=encoder_outputs, labels=batch["labels"])
        else:
            # do the full forward pass for the teacher model (encoder + decoder)
            teacher_outputs = teacher_model(**batch)

    # CE (data) loss
    ce_loss = student_outputs.loss
    # rescale distribution by temperature to ensure gradients scale correctly
    teacher_distribution = nn.functional.softmax(teacher_outputs.logits / temperature, dim=-1)
    # log softmax of student predictions for numerical stability
    student_distribution = nn.functional.log_softmax(student_outputs.logits / temperature, dim=-1)
    # KL-divergence loss (scaled by temperature)
    kl_loss = kl_divergence(teacher_distribution, student_distribution, batch["labels"]) * temperature**2

    # use Distil-Whisper formulation (fix weight of CE loss and tune KL weight)
    loss = 0.8 * ce_loss + training_args.kl_weight * kl_loss
    metrics = {"loss": loss, "ce_loss": ce_loss, "kl_loss": kl_loss}
    return loss, metrics

# Define eval fn
def eval_step(batch):
    student_model.eval()
    teacher_model.eval()

    with torch.no_grad():
        student_outputs = student_model(**batch)
        if share_hidden_states:
            encoder_outputs = BaseModelOutput(student_outputs.encoder_last_hidden_state)
            teacher_outputs = teacher_model(encoder_outputs=encoder_outputs, labels=batch["labels"])
        else:
            teacher_outputs = teacher_model(**batch)

    # CE (data) loss
    ce_loss = student_outputs.loss

    # log softmax / softmax for numerical stability
    student_distribution = nn.functional.log_softmax(student_outputs.logits, dim=-1)
    teacher_distribution = nn.functional.softmax(teacher_outputs.logits, dim=-1)
    # temperature is always 1 for eval
    kl_loss = kl_divergence(teacher_distribution, student_distribution, batch["labels"])

    # use Distil-Whisper formulation (fix weight of CE loss and tune KL weight)
    loss = 0.8 * ce_loss + training_args.kl_weight * kl_loss
    metrics = {"loss": loss, "ce_loss": ce_loss, "kl_loss": kl_loss}
    return metrics

def generate_step(batch):
    student_model.eval()
    output_ids = accelerator.unwrap_model(student_model).generate(batch["input_features"], **gen_kwargs)
    output_ids = accelerator.pad_across_processes(output_ids, dim=1, pad_index=tokenizer.pad_token_id)
    return output_ids

In [33]:
logger.info("***** Running training *****")
logger.info(f"  Num examples = {total_train_steps * train_batch_size * gradient_accumulation_steps}")
logger.info("  Instantaneous batch size per device =" f" {training_args.per_device_train_batch_size}")
logger.info("  Gradient accumulation steps =" f" {gradient_accumulation_steps}")
logger.info(
    f"  Total train batch size (w. parallel & distributed) = {train_batch_size * gradient_accumulation_steps}"
)
logger.info(f"  Total optimization steps = {total_train_steps}")

# ======================== Training ================================
train_time = 0
train_start = time.time()
steps_trained_progress_bar = tqdm(
    range(total_train_steps), desc="Train steps ... ", position=0, disable=not accelerator.is_local_main_process
)
continue_training = True
epochs_trained = 0
cur_step = 0

checkpoint = None
if training_args.resume_from_checkpoint is not None:
    checkpoint = training_args.resume_from_checkpoint
elif last_checkpoint is not None:
    checkpoint = last_checkpoint

if checkpoint is not None:
    accelerator.load_state(checkpoint)
    # Find num steps and epoch from saved state string pattern
    pattern = r"checkpoint-(\d+)-epoch-(\d+)"
    match = re.search(pattern, checkpoint)
    cur_step = int(match.group(1))
    epochs_trained = int(match.group(2))

    logger.info("  Continuing training from checkpoint, will skip to saved global_step")
    logger.info(f"  Continuing training from epoch {epochs_trained}")
    logger.info(f"  Continuing training from global step {cur_step}")

    steps_trained_progress_bar.update(cur_step)

    for epoch in range(0, epochs_trained):
        vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(training_args.seed)

    if not data_args.streaming and training_args.max_steps < 0:
        # we know exactly the number of steps per epoch, so can skip through the required number of batches
        resume_step = (cur_step - epochs_trained * steps_per_epoch) * gradient_accumulation_steps
    else:
        # Currently we don't know how many steps we've taken in the current epoch
        # So we just shuffle the dataset one extra time and start from a fresh epoch
        # This is "good enough" for our purposes but not fully correct
        resume_step = None
        vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(training_args.seed)
else:
    resume_step = None

04/16/2024 18:26:46 - INFO - __main__ - ***** Running training *****
04/16/2024 18:26:46 - INFO - __main__ -   Num examples = 240000
04/16/2024 18:26:46 - INFO - __main__ -   Instantaneous batch size per device = 24
04/16/2024 18:26:46 - INFO - __main__ -   Gradient accumulation steps = 1
04/16/2024 18:26:46 - INFO - __main__ -   Total train batch size (w. parallel & distributed) = 24
04/16/2024 18:26:46 - INFO - __main__ -   Total optimization steps = 10000
Train steps ... :   0%|          | 0/10000 [00:00<?, ?it/s]

In [34]:
for epoch in range(epochs_trained, num_epochs):
    vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(training_args.seed)
    train_dataloader = DataLoader(
        vectorized_datasets["train"],
        collate_fn=data_collator,
        batch_size=per_device_train_batch_size,
        num_workers=dataloader_num_workers,
        pin_memory=training_args.dataloader_pin_memory,
    )
    train_dataloader = accelerator.prepare(train_dataloader)
    if hasattr(train_dataloader, "dataset") and isinstance(train_dataloader.dataset, IterableDataset):
        train_dataloader.dataset.set_epoch(epoch)

    if resume_step is not None:
        # Skip the first N batches in the dataloader when resuming from a checkpoint
        train_dataloader = accelerator.skip_first_batches(train_dataloader, resume_step)
        resume_step = None

    for batch in train_dataloader:
        with accelerator.accumulate(student_model):
            loss, train_metric = train_step(batch, temperature=training_args.temperature)
            accelerator.backward(loss)
            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(student_model.parameters(), training_args.max_grad_norm)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Check if the accelerator has performed an optimization step behind the scenes
        if accelerator.sync_gradients:
            steps_trained_progress_bar.update(1)
            cur_step += 1

            if cur_step % training_args.logging_steps == 0:
                steps_trained_progress_bar.write(
                    f"Step... ({cur_step} / {total_train_steps} | Loss:"
                    f" {train_metric['loss']}, Learning Rate:"
                    f" {lr_scheduler.get_last_lr()[0]})"
                )
                log_metric(
                    accelerator,
                    metrics=train_metric,
                    learning_rate=lr_scheduler.get_last_lr()[0],
                    train_time=train_time + time.time() - train_start,
                    step=cur_step,
                    epoch=epoch,
                    prefix="train",
                )

            # save checkpoint and weights after each save_steps and at the end of training
            if (cur_step % training_args.save_steps == 0) or cur_step == total_train_steps:
                intermediate_dir = os.path.join(training_args.output_dir, f"checkpoint-{cur_step}-epoch-{epoch}")
                accelerator.save_state(output_dir=intermediate_dir)
                accelerator.wait_for_everyone()
                if accelerator.is_main_process:
                    rotate_checkpoints(training_args.save_total_limit, output_dir=training_args.output_dir)

                    if cur_step == total_train_steps:
                        student_model = accelerator.unwrap_model(student_model)
                        student_model.save_pretrained(training_args.output_dir)

                    if training_args.push_to_hub:
                        repo.push_to_hub(
                            commit_message=f"Saving train state of step {cur_step}",
                            blocking=False,
                        )

            if training_args.do_eval and (cur_step % eval_steps == 0 or cur_step == total_train_steps):
                train_time += time.time() - train_start
                student_model.eval()
                # ======================== Evaluating ==============================
                for eval_split in ['eval']:
                    eval_metrics = []
                    eval_preds = []
                    eval_labels = []
                    eval_start = time.time()

                    validation_dataloader = DataLoader(
                        vectorized_datasets[eval_split],
                        collate_fn=data_collator,
                        batch_size=per_device_eval_batch_size,
                        drop_last=False,
                        num_workers=dataloader_num_workers,
                        pin_memory=training_args.dataloader_pin_memory,
                    )
                    validation_dataloader = accelerator.prepare(validation_dataloader)

                    for batch in tqdm(
                        validation_dataloader,
                        desc=f"Evaluating {eval_split}...",
                        position=2,
                        disable=not accelerator.is_local_main_process,
                    ):
                        # Model forward
                        eval_metric = eval_step(batch)
                        eval_metric = accelerator.gather_for_metrics(eval_metric)
                        eval_metrics.append(eval_metric)

                        # generation
                        if training_args.predict_with_generate:
                            generated_ids = generate_step(batch)
                            # Gather all predictions and targets
                            generated_ids, labels = accelerator.gather_for_metrics(
                                (generated_ids, batch["labels"])
                            )
                            eval_preds.extend(generated_ids)
                            eval_labels.extend(labels)

                    eval_time = time.time() - eval_start
                    # normalize eval metrics
                    eval_metrics = {
                        key: torch.mean(torch.stack([d[key] for d in eval_metrics])) for key in eval_metrics[0]
                    }

                    # compute WER metric
                    wer_desc = ""
                    if training_args.predict_with_generate:
                        wer_metric, pred_str, label_str, norm_pred_str, norm_label_str = compute_metrics(
                            eval_preds, eval_labels
                        )
                        eval_metrics.update(wer_metric)
                        wer_desc = " ".join([f"Eval {key}: {value} |" for key, value in wer_metric.items()])
                        log_pred(
                            accelerator,
                            pred_str,
                            label_str,
                            norm_pred_str,
                            norm_label_str,
                            step=cur_step,
                            prefix=eval_split,
                        )

                    # Print metrics and update progress bar
                    steps_trained_progress_bar.write(
                        f"Eval results for step ({cur_step} / {total_train_steps} | Eval Loss: {eval_metrics['loss']} |"
                        f" {wer_desc})"
                    )

                    log_metric(
                        accelerator,
                        metrics=eval_metrics,
                        train_time=eval_time,
                        step=cur_step,
                        epoch=epoch,
                        prefix=eval_split,
                    )

                # flush the train metrics
                train_start = time.time()

            # break condition
            if cur_step == total_train_steps:
                continue_training = False
                break

    if not continue_training:
        break

accelerator.end_training()


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
f:\anaconda\envs\disenv\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Train steps ... :   0%|          | 25/10000 [01:02<4:03:11,  1.46s/it]

Step... (25 / 10000 | Loss: 37.562034606933594, Learning Rate: 6.800000000000001e-07)


Train steps ... :   0%|          | 50/10000 [01:39<4:05:28,  1.48s/it]

Step... (50 / 10000 | Loss: 14.571999549865723, Learning Rate: 1.6800000000000002e-06)


Train steps ... :   1%|          | 75/10000 [02:16<4:07:14,  1.49s/it]

...

Train steps ... : 100%|██████████| 10000/10000 [5:34:55<00:00,  1.64s/it]04/17/2024 00:01:41 - INFO - accelerate.accelerator - Saving current state to ./result_distiling_3\checkpoint-10000-epoch-20
04/17/2024 00:01:41 - WARNING - accelerate.utils.other - Removed shared tensor {'proj_out.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


Step... (10000 / 10000 | Loss: 0.405915766954422, Learning Rate: 1e-05)


04/17/2024 00:01:43 - INFO - accelerate.checkpointing - Model weights saved in result_distiling_3\checkpoint-10000-epoch-20\model.safetensors
04/17/2024 00:01:43 - WARNING - accelerate.utils.other - Removed shared tensor {'proj_out.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
04/17/2024 00:01:44 - INFO - accelerate.checkpointing - Model weights saved in result_distiling_3\checkpoint-10000-epoch-20\model_1.safetensors
04/17/2024 00:01:45 - INFO - accelerate.checkpointing - Optimizer state saved in result_distiling_3\checkpoint-10000-epoch-20\optimizer.bin
04/17/2024 00:01:45 - INFO - accelerate.checkpointing - Scheduler state saved in result_distiling_3\checkpoint-10000-epoch-20\scheduler.bin
04/17/2024 00:01:45 - INFO - accelerate.checkpointing - Sampler state for dataloader 0 saved in result_distiling_3\checkpoint-10000-epoch-20\sampler.bin
04/17/2024 00:01:45 - INFO - accelerate.checkpointing - Sampler state for d

Eval results for step (10000 / 10000 | Eval Loss: 15.371350288391113 | )


eval/ce_loss,▆▃▂▃▆▅▅▁▃█▄▄▅▆▆▅▇▇▇█
eval/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
eval/kl_loss,▅▁▄▃▆▆▅▄▄▆▇▅▆▇▇▆▇███
eval/loss,▅▁▃▂▆▅▅▂▃▇▅▅▆▇▆▆▇█▇█
eval/time,▁▂▃▂▁▂▁▁▂▂▂▃▃▄▆▆▅▆▇█
train/ce_loss,█▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/kl_loss,█▅▅▄▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁███████████████████████████████████████
train/loss,█▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/time,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [37]:
accelerator.end_training()

In [35]:
accelerator